# Reverse engineering of realized TDI stencil
Some aPCI studies will may want as a diagnostic, comparison, or reference, a representation of stencil that is applied by the TDI algorithm, or some approximant.  We had tools that could be applied for this from ground up in the old pylisa code, but it would be most valuable to be able to match with the actual stencils used in pyTDI sometimes.   Getting all the conventions and details right can be hard.  I did not notice a straightforward way to get this stencil info from the pyTDI code functions, because that isn't really how the calculation is organized at least at top level.  Fortunately it should be straightforward to extract this from the pyTDI code as a black box by probing it's impulse response.

In [1]:
from pytdi.michelson import X1, Y1, Z1, X2, Y2, Z2
from pytdi import Data
import h5py
import numpy as np
import copy
import pcipy

datadir = "/data/jgbaker/software/pylisa/data/"
orbits = datadir+"keplerian-orbits.h5"
workdir = datadir+"/simulations/"
noise_file_base = "2025-07-01_19h38_laser_tm_oms_"
dtpath = "2024-04-10_18h14_"

In [2]:
##This is the code for computing TDI as we do it in noise_simulation:
if 0:  #the code is just to look at
    data_noise = Data.from_instrument(instr)
    if args.tdi == '2':
        X, Y, Z = X2, Y2, Z2
    else:
        X, Y, Z = X1, Y1, Z1
    # Build other 2.0 Michelson variables
    X_data = X.build(**data_noise.args)
    Y_data = Y.build(**data_noise.args)
    Z_data = Z.build(**data_noise.args)
    # Apply TDI 2.0
    x_noise = X_data(data_noise.measurements)
    y_noise = Y_data(data_noise.measurements)
    z_noise = Z_data(data_noise.measurements)

In [3]:
# If we use a file
simpath = workdir + dtpath + 'measurements_4Hz.h5'
#simpath = workdir + noise_file_base + 'measurements_4Hz.h5'
# load hdf5 file to read data attrs
sim = h5py.File(simpath, 'r')
# load data
data_noise = Data.from_instrument(simpath)

fs = data_noise.fs

# Build other 2.0 Michelson variables
X_data_op = X2.build(**data_noise.args)
Y_data_op = Y2.build(**data_noise.args)
Z_data_op = Z2.build(**data_noise.args)

X_n_orig=X_data_op(data_noise.measurements)



You are using a measurement file in a version that might not be fully supported


In [4]:

#Test that we can work-with and reconstruct measurements and TDI for shorter series
#REF measurements = {f'isi_{link}': dset[:,ilink] for ilink, link in enumerate(links)}
mosas_order = ['12', '23', '31', '13', '21', '32']

n=20000
#make modified version of the data_noise args
newargs=copy.deepcopy(data_noise.args)
for k in newargs['delays'].keys():
    #print(k,newargs['delays'][k].shape,newargs['delay_derivatives'][k].shape)
    newargs['delays'][k]=newargs['delays'][k][:n]
    newargs['delay_derivatives'][k]=newargs['delay_derivatives'][k][:n]
#X_data_op_short = X2.build(**newargs)
XYZ_ops=[chan.build(**newargs) for chan in [X2,Y2,Z2]]
display(newargs)

measurements_short=copy.deepcopy(data_noise.measurements)
for k in measurements_short.keys():
    #print(k,measurements[k].shape)
    measurements_short[k]=measurements_short[k][:n]
    
X_n_short=XYZ_ops[0](measurements_short)

print(X_n_orig[:n]-X_n_short,np.std(X_n_orig[:n]-X_n_short))
print(X_n_short)

{'delays': {'d_12': array([-8.70607664e-27, -9.41529405e-10,  3.98050548e-08, ...,
          8.33243894e+00,  8.33243894e+00,  8.33243894e+00]),
  'd_23': array([-8.67514819e-27, -9.38184610e-10,  3.96636468e-08, ...,
          8.30282194e+00,  8.30282194e+00,  8.30282194e+00]),
  'd_31': array([-8.70607667e-27, -9.41529408e-10,  3.98050549e-08, ...,
          8.33240702e+00,  8.33240702e+00,  8.33240702e+00]),
  'd_13': array([-8.70521053e-27, -9.41435739e-10,  3.98010948e-08, ...,
          8.33157805e+00,  8.33157805e+00,  8.33157805e+00]),
  'd_32': array([-8.67686796e-27, -9.38370597e-10,  3.96715098e-08, ...,
          8.30446790e+00,  8.30446790e+00,  8.30446790e+00]),
  'd_21': array([-8.70521050e-27, -9.41435736e-10,  3.98010947e-08, ...,
          8.33160997e+00,  8.33160997e+00,  8.33160997e+00])},
 'fs': 4.0,
 'delay_derivatives': {'d_12': array([-3.76611762e-09,  7.96101096e-08, -1.07046185e-06, ...,
          3.19808890e-09,  3.19808535e-09,  3.19808890e-09]),
  'd_23': a

[0. 0. 0. ... 0. 0. 0.] 0.0
[ 7.91700037e-36  3.09996986e-17 -4.25476397e-15 ... -3.69809798e+01
  3.75295520e+01 -3.28909303e+01]


In [5]:
#now let's hack that process to get the zero-responses
measurements_z=copy.deepcopy(measurements_short)
for link in mosas_order:
    name=f'isi_{link}'
    data=np.zeros((n,))
    #print(name,data.shape,'\n',data)
    measurements_z[name]=data
print('computing TDI')
XYZ_z=np.array([op(measurements_z) for op in XYZ_ops])
print(XYZ_z)

computing TDI
[[ 1.72191555e-40 -2.64697796e-23  2.85873620e-21 ...  1.43251430e+02
   1.88394944e+02 -1.67589604e+02]
 [-1.14794370e-40  1.04224757e-22 -9.36633151e-20 ...  1.71342445e+02
   1.87736246e+02  5.01058165e+01]
 [-2.29588740e-41  6.28657266e-23  4.23599192e-21 ...  2.75048931e+01
  -1.55602416e+02  3.12818294e+01]]


In [6]:
#now let's hack that process to get the single-point-impulse-responses
i0=1000
XYZ_1={}
for link in mosas_order:
    print(link)
    measurements_1=copy.deepcopy(measurements_z)
    name=f'isi_{link}'
    #print(name,data.shape,'\n',data)
    measurements_1[name][i0]=1
    XYZ_z=np.array([op(measurements_1) for op in XYZ_ops])
    XYZ_1[link]=np.array([op(measurements_z) for op in XYZ_ops])-XYZ_z
display(XYZ_1)

12
23
31
13
21
32


{'12': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '23': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '31': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '13': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '21': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '32': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])}

In [7]:
minmaxs=[]
for link in mosas_order:
    for i in range(3):
        nz=np.nonzero(XYZ_1[link][i])[0]
        minmax = [min(nz),max(nz)] if len(nz)>0 else ''
        print(link,'XYZ'[i],len(nz),minmax)
        if len(nz)>0:minmaxs+=[minmax]
first=min([minmax[0] for minmax in minmaxs])
last=max([minmax[1] for minmax in minmaxs])
length=last+1-first
print('length,first,last',length,first,last)
        

12 X 97 [1000, 1215]
12 Y 128 [1018, 1248]
12 Z 0 
23 X 0 
23 Y 97 [1000, 1215]
23 Z 128 [1018, 1248]
31 X 128 [1018, 1249]
31 Y 0 
31 Z 97 [1000, 1215]
13 X 97 [1000, 1215]
13 Y 0 
13 Z 128 [1018, 1248]
21 X 128 [1018, 1249]
21 Y 97 [1000, 1215]
21 Z 0 
32 X 0 
32 Y 128 [1018, 1248]
32 Z 97 [1000, 1215]
length,first,last 250 1000 1249


In [8]:
stencils=np.array([XYZ_1[link][:,first:last+1] for link in mosas_order]).transpose((1,0,2))
print(stencils.shape)

(3, 6, 250)


In [9]:
###Now we reorganize pack that all into a neat bundle
def compute_TDI_stencil(noise_data,i0=0,nleft=500,nright=1000):

    assert i0-nleft>=0, "out of range"
    mosas_order = ['12', '23', '31', '13', '21', '32']

    #make modified version of the data_noise args
    newargs=copy.deepcopy(noise_data.args)
    for k in newargs['delays'].keys():
        #print(k,newargs['delays'][k].shape,newargs['delay_derivatives'][k].shape)
        newargs['delays'][k]=newargs['delays'][k][i0-nleft:i0+nright]
        newargs['delay_derivatives'][k]=newargs['delay_derivatives'][k][i0-nleft:i0+nright]
    XYZ_ops=[chan.build(**newargs) for chan in [X2,Y2,Z2]]
    #display(newargs)

    #Compute the zero-response for the relevant stretch of data
    measurements_z=copy.deepcopy(noise_data.measurements)
    for k in measurements_z.keys():
        #print(k,measurements[k].shape)
        measurements_z[k]=measurements_z[k][i0-nleft:i0+nright]

    for link in mosas_order:
        name=f'isi_{link}'
        data=np.zeros((nleft+nright,))
        measurements_z[name]=data
    #print('computing TDI')
    XYZ_z=np.array([op(measurements_z) for op in XYZ_ops])
    #print(XYZ_z)
    
    #now get the single-point-impulse-responses
    XYZ_1={}
    minmaxs=[]
    for link in mosas_order:
        #print(link)
        measurements_1=copy.deepcopy(measurements_z)
        name=f'isi_{link}'
        #print(name,data.shape,'\n',data)
        measurements_1[name][nleft]=1
        XYZ_z=np.array([op(measurements_1) for op in XYZ_ops])
        XYZ_1[link]=np.array([op(measurements_z) for op in XYZ_ops])-XYZ_z
        for i in range(3):
            nz=np.nonzero(XYZ_1[link][i])[0]
            minmax = [min(nz),max(nz)] if len(nz)>0 else ''
            #print(link,'XYZ'[i],len(nz),minmax)
            if len(nz)>0:minmaxs+=[minmax]
    #display(XYZ_1)
    first=min([minmax[0] for minmax in minmaxs])
    last=max([minmax[1] for minmax in minmaxs])
    length=last+1-first
    print('length,first,last',length,first,last)
    
    stencil=np.zeros((3,6,length))
    for ilink in range(len(mosas_order)):
        link=mosas_order[ilink]
        for i in range(3):
            stencil[:,ilink,:]=XYZ_1[link][:,first:last+1]
        #print('shape change:',XYZ_1[link].shape,stencil[link].shape)
    ioff=nleft-first
    #print('ioff',ioff)
    #display(stencil)
    return ioff, stencil    


In [10]:
#Read data from file
simpath = workdir + noise_file_base + 'measurements_4Hz.h5'
# load hdf5 file to read data attrs
sim = h5py.File(simpath, 'r')
# load data
data_noise = Data.from_instrument(simpath)

You are using a measurement file in a development version
You are using a measurement file in a version that might not be fully supported


In [11]:
compute_TDI_stencil(data_noise,i0=1000,nleft=1000,nright=19000)

length,first,last 250 1000 1249


(0,
 array([[[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           2.83373538e-08, -1.76078174e-09,  5.31485966e-11],
         [-1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -2.85622264e-08,  1.77442416e-09, -5.36033440e-11],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
 
        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -1.10287601e-09,  3.31965566e-11,  0.00000000e+00],
         [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.0

In [12]:
ioff,s1000=compute_TDI_stencil(data_noise,i0=1000,nleft=100,nright=500)

length,first,last 250 100 349


In [13]:
ioff,s2000=compute_TDI_stencil(data_noise,i0=2000,nleft=100,nright=500);print(ioff)
ioff,s4000=compute_TDI_stencil(data_noise,i0=4000,nleft=100,nright=500);print(ioff)
ioff,s8000=compute_TDI_stencil(data_noise,i0=8000,nleft=100,nright=500);print(ioff)
ioff,s16000=compute_TDI_stencil(data_noise,i0=16000,nleft=100,nright=500);print(ioff)
ioff,s32000=compute_TDI_stencil(data_noise,i0=32000,nleft=100,nright=500);print(ioff)
ioff,s64000=compute_TDI_stencil(data_noise,i0=64000,nleft=100,nright=500);print(ioff)
ioff,s128000=compute_TDI_stencil(data_noise,i0=128000,nleft=100,nright=500);print(ioff)
ioff,s256000=compute_TDI_stencil(data_noise,i0=256000,nleft=100,nright=500);print(ioff)
ioff,s512000=compute_TDI_stencil(data_noise,i0=512000,nleft=100,nright=500);print(ioff)

length,first,last 250 100 349
0
length,first,last 250 100 349
0
length,first,last 250 100 349
0
length,first,last 250 100 349
0
length,first,last 250 100 349
0
length,first,last 250 100 349
0
length,first,last 250 100 349
0
length,first,last 250 100 349
0
length,first,last 250 100 349
0


In [14]:
def ip(a,b):
    return np.sum(a*b)
def norm(a):
    return ip(a,a)
def ac(a,b):
    return ip(a,b)/np.sqrt(norm(a)*norm(b))

In [15]:
print(s1000[0].shape)

(6, 250)


In [16]:
def ac_matrix(a,b): return [[ac(a[i],b[j]) for i in range(3)] for j in range(3)]
display(ac_matrix(s1000,s1000))
display(ac_matrix(s2000,s2000))
display(ac_matrix(s1000,s2000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.999999999978185, 0.0, 0.0],
 [0.0, 0.9999999999314465, 0.0],
 [0.0, 0.0, 0.9999999999315501]]

In [17]:
display(ac_matrix(s1000,s1000))
display(ac_matrix(s4000,s4000))
display(ac_matrix(s1000,s4000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.9999999998036644, 0.0, 0.0],
 [0.0, 0.9999999993829175, 0.0],
 [0.0, 0.0, 0.9999999993840522]]

In [18]:
display(ac_matrix(s1000,s1000))
display(ac_matrix(s8000,s8000))
display(ac_matrix(s1000,s8000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.9999999989310613, 0.0, 0.0],
 [0.0, 0.9999999966392243, 0.0],
 [0.0, 0.0, 0.9999999966476086]]

In [19]:
display(ac_matrix(s1000,s1000))
display(ac_matrix(s16000,s16000))
display(ac_matrix(s1000,s16000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.9999999950916093, 0.0, 0.0],
 [0.0, 0.9999999845577234, 0.0],
 [0.0, 0.0, 0.999999984616496]]

In [20]:
display(ac_matrix(s1000,s1000))
display(ac_matrix(s32000,s32000))
display(ac_matrix(s1000,s32000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.9999999790357329, 0.0, 0.0],
 [0.0, 0.9999999339576379, 0.0],
 [0.0, 0.0, 0.9999999343818382]]

In [21]:
display(ac_matrix(s1000,s1000))
display(ac_matrix(s64000,s64000))
display(ac_matrix(s1000,s64000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.9999999134162745, 0.0, 0.0],
 [0.0, 0.9999997265236256, 0.0],
 [0.0, 0.0, 0.999999729706047]]

In [22]:
display(ac_matrix(s1000,s1000))
display(ac_matrix(s128000,s128000))
display(ac_matrix(s1000,s128000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.9999996481493395, 0.0, 0.0],
 [0.0, 0.9999988828282903, 0.0],
 [0.0, 0.0, 0.9999989073861953]]

In [23]:
display(ac_matrix(s1000,s1000))
display(ac_matrix(s256000,s256000))
display(ac_matrix(s1000,s256000))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

[[0.9999985815471926, 0.0, 0.0],
 [0.0, 0.9999954488702867, 0.0],
 [0.0, 0.0, 0.9999956415963631]]

In [24]:
#Compare with code in tdi_filter:
import importlib
importlib.reload(pcipy)
import pcipy.tdi_filter
importlib.reload(pcipy.tdi_filter)
from pcipy.tdi_filter import DeducedTDIFilter as TDIFilter
ioff,sp1000=TDIFilter.compute_stencil_from_measurements(data_noise,i0=1000,nleft=400,nright=600)
tdifilt=TDIFilter(data_noise,i0=1000)
st1000=np.transpose(tdifilt.stencil_compts,[0,2,1])

In [25]:
print(np.sort(sp1000-st1000))
print(np.mean(sp1000-st1000))

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
0.0


In [26]:
from pytdi.intervar import ETA_SET
skip=1000
mosas_order=['12', '23', '31', '13', '21', '32']
def build_data_vector(data_noise, skip=300, dtype=np.float64):
    central_freq = 281600000000000.0
    # Conventions 1, 2, 3, 1p, 2p, 3p
    delays_order = ['23', '31', '12', '32', '13', '21']
    mosas_order = ['12', '23', '31', '13', '21', '32']
    # Form intermediary variables for full data
    ETA_data_set = {key: ETA_SET[key].build(**data_noise.args) for key in ETA_SET.keys()}
    eta_noise_set = {key: ETA_data_set[key](data_noise.measurements) for key in ETA_data_set.keys()}
    # Form the measurement vector for moving arms containing all noises
    y = np.array([eta_noise_set[f'eta_{mosa}'] / central_freq for mosa in mosas_order], dtype=dtype).T
    y_full = y[skip:, :]
    del y

    return y_full

y_full = build_data_vector(data_noise, skip=skip)
print(y_full.shape)

(1035800, 6)


In [27]:
importlib.reload(pcipy)
import pcipy.data
importlib.reload(pcipy.data)
importlib.reload(pcipy.filter)
from pcipy.data import TimeData
y_td=TimeData(y_full, dt=1/fs,names=mosas_order)

In [28]:
XYZ=tdifilt.apply_filter(y_td)

Wrong count of channels.


AssertionError: Data check was:False